In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

import random

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score


import itertools
import glob

random.seed(1)
np.random.seed(seed=1)

In [ ]:
# ------------------------------------------------------------------------------
# Read data
# ------------------------------------------------------------------------------
train_files = sorted(glob.glob('../input/wifi-over-500/train/*_train.csv'))
test_files = sorted(glob.glob('../input/wifi-over-500/test/*_test.csv'))

In [ ]:
def all_get(file_name_train, file_name_test):
    
    file_path = file_name_train
    df_train = pd.read_csv(file_path)

    test_file_path = file_name_test
    df_test = pd.read_csv(test_file_path)


    temp_df_train = df_train.copy()
    temp_df_test = df_test.copy()




    #target = temp_df_train[['x','y','f']].values / 1000
    #target = np.asarray(target).astype(np.float64)

    target = []
    target.append(temp_df_train['x'])
    target.append(temp_df_train['y'])
    target.append(temp_df_train['f'])

    #target = np.asarray(target).astype(np.float64)

    path_train_temp = temp_df_train['path'].map(lambda x: int(x, 16))
    path_train_temp = path_train_temp%100
    temp_df_train['path'] = path_train_temp
    temp_df_train = temp_df_train
    temp_df_train.pop('x')
    temp_df_train.pop('y')
    temp_df_train.pop('f')
    temp_df_train.pop('path')
    temp_df_train = temp_df_train.drop(temp_df_train.columns[[0]], axis=1)
    #train_data = np.asarray(temp_df_train).astype(np.float64)

    #path_test_temp = temp_df_test['site_path_timestamp'].map(lambda x: int(x, 16))
    temp_df_test.pop('site_path_timestamp')
    temp_df_test = temp_df_test.drop(temp_df_test.columns[[0]], axis=1)
    #temp_df_test /=1000
    #temp_df_test = np.asarray(temp_df_test).astype(np.float64)



    MODEL_PARAM = {"batch_size":1024,
                  "verbose":0,
                  "epochs":100}
    callbacks_list=[EarlyStopping(monitor="loss", min_delta=.00000001, patience=5,mode='auto')]


    def build_model() -> Model:
        # モデル定義
        model = Sequential()
        model.add(Dense(512,activation=tf.nn.relu))
        model.add(Dropout(0.1))
        model.add(Dense(32, activation=tf.nn.relu))
        model.add(Dense(1))
        model.compile(optimizer="Adam", loss='mse', metrics=["mae"])

        return model



    scores = []
    model_list = []

    for i in range(2):

        train_x = temp_df_train
        train_y = target[i]


        # KFoldクラスを用いてクロスバリデーションの分割を行う
        kf = KFold(n_splits=4, shuffle=True, random_state=71)
        for tr_idx, va_idx in kf.split(train_x):
            tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
            tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

            tr_x = np.asarray(tr_x).astype(np.float64)
            tr_y = np.asarray(tr_y).astype(np.float64)
            va_x = np.asarray(va_x).astype(np.float64)
            va_y = np.asarray(va_y).astype(np.float64)


            # 学習の実行、バリデーションデータの予測値の出力、スコアの計算を行う
            model = build_model()
            history = model.fit(tr_x, tr_y,
                              validation_data=(va_x, va_y),
                              callbacks=callbacks_list,
                              **MODEL_PARAM)
            va_pred = model.predict(va_x)
            score = np.abs(va_y-va_pred)
            scores.append(score)
        model_list.append(model)



    test_predictions_X = model_list[0].predict(temp_df_test)
    test_predictions_Y = model_list[1].predict(temp_df_test)


    train_x = temp_df_train
    train_y = target[2]

    model = tf.keras.Sequential()
    # ユニット数が64の全結合層をモデルに追加します：
    #全結合、入力64次元、出力64次元、活性化関数relu
    model.add(layers.Dense(128, activation='relu'))
    #全結合、入力64次元、出力64次元、活性化関数relu
    model.add(layers.Dense(64, activation='relu'))
    #全結合、入力64次元、出力2次元、活性化関数softmax
    model.add(layers.Dense(15,activation="softmax"))





    # 定数
    LOSS = 'categorical_crossentropy'  # 損失関数：多クラス分類用の交差エントロピー
    METRICS = ['accuracy']                    # 評価関数：正解率
    OPTIMIZER = tf.keras.optimizers.Adam      # 最適化：
    LEARNING_RATE = 0.001                     # 学習率： 0.001（学習率の調整）


    # 学習方法を定義する
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    train_y_onehot = to_categorical(target[2])




    col_name = [-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9]
    f = []
    #df_dlabel = pd.DataFrame(np.arange(-5,10).reshape(1,15))
    for i, temp_tar in enumerate(target[2]):
        a = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        a[temp_tar+5] = 1
        f.append(a)
        #df_dlabel.append




    train_y = np.array(f)




    model_list_f = []
    # 定数（ミニバッチ学習時に必要となるもの）
    BATCH_SIZE = 96   # バッチサイズ： 96
    EPOCHS = 100      # エポック数： 100

    # 早期終了
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    # 学習する
    hist = model.fit(x=train_x,                          # 訓練用データ
                     y=train_y,                          # 訓練用ラベル
                     validation_split=0.2,               # 精度検証用 20％
                     batch_size=BATCH_SIZE,              # バッチサイズ
                     epochs=EPOCHS,                      # エポック数
                     verbose=0,                          # 実行状況表示
                     callbacks=[                         # コールバック
                       es #早期終了
                     ])
    model_list_f.append(model)



    len(model_list_f)

    test_predictions_F = model_list_f[0].predict(temp_df_test)

    np.set_printoptions(threshold=np.inf)




    pred_class_F = np.argmax(test_predictions_F, axis=-1)-5

    return pred_class_F, test_predictions_X,test_predictions_Y



In [ ]:
predictions = list()
for temp_train_file,temp_test_file in tqdm(zip(train_files,test_files)):
    print(temp_train_file)
    predictions.append(all_get(temp_train_file,temp_test_file))



In [ ]:
pre_f = []
pre_x = []
pre_y = []

for p in predictions:
    pre_f.extend(p[0].tolist())
    
    for x in p[1]:
        x = float(x)
        pre_x.append(x)
        
    for y in p[2]:
        y = float(y)
        pre_y.append(y)
        

pf = pd.DataFrame(data=pre_f)
px = pd.DataFrame(data=pre_x)
py = pd.DataFrame(data=pre_y)


subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv', index_col=0)
subm['floor'] = pre_f
subm['x'] = pre_x
subm['y'] = pre_y

subm.to_csv("submission.csv")